# Dev simple

In [ ]:
#@title 🛠️ Setup
!nvidia-smi -L || true

import os, sys, random, numpy as np, torch, json, time, platform, math
print("Python:", sys.version)
print("CUDA available:", torch.cuda.is_available())

# Qwen3 requires Transformers >= 4.51
try:
    get_ipython().run_line_magic("uv", "pip -q install transformers==4.51.3 accelerate==1.4.0 peft==0.14.0 datasets==3.3.2 evaluate==0.4.3 sentencepiece protobuf tqdm matplotlib > /dev/null")
except Exception:
    get_ipython().run_line_magic("pip", "-q install transformers==4.51.3 accelerate==1.4.0 peft==0.14.0 datasets==3.3.2 evaluate==0.4.3 sentencepiece protobuf tqdm matplotlib > /dev/null")

import transformers, datasets, peft, accelerate, matplotlib
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("PEFT:", peft.__version__)
print("matplotlib:", matplotlib.__version__)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "Please connect a GPU (A100+ recommended)."

def print_header():
    print("== Environment ==")
    print(dict(
        python=sys.version,
        torch=torch.__version__,
        transformers=transformers.__version__,
        accelerate=accelerate.__version__,
        peft=peft.__version__,
        cuda=torch.version.cuda if torch.cuda.is_available() else "cpu",
        device_name=torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu",
        platform=platform.platform(),
        seed=SEED
    ))
print_header()

import os, sys, time, json, random, platform
from dataclasses import dataclass
from typing import Optional, List, Dict

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

import pandas as pd
from IPython.display import display

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import re

# --------------------------
# Reproducibility & device
# --------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "A CUDA GPU is required."

GPU 0: NVIDIA RTX 4000 Ada Generation Laptop GPU (UUID: GPU-2e6f8d70-aa40-c48f-e928-5c9265e18c5f)
Python: 3.12.12 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 20:05:38) [MSC v.1929 64 bit (AMD64)]
CUDA available: True
Note: you may need to restart the kernel to use updated packages.
Transformers: 4.51.3
Accelerate: 1.4.0
PEFT: 0.14.0
matplotlib: 3.10.7
== Environment ==
{'python': '3.12.12 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 20:05:38) [MSC v.1929 64 bit (AMD64)]', 'torch': '2.6.0+cu124', 'transformers': '4.51.3', 'accelerate': '1.4.0', 'peft': '0.14.0', 'cuda': '12.4', 'device_name': 'NVIDIA RTX 4000 Ada Generation Laptop GPU', 'platform': 'Windows-10-10.0.19045-SP0', 'seed': 42}


El sistema no puede encontrar la ruta especificada.


In [ ]:
# --------------------------
# Config
# --------------------------
@dataclass
class Config:
    # Models
    student_id: str = "Qwen/Qwen3-0.6B-Base"  # Modelo estudiante pequeño (0.6B parámetros) que será entrenado
    # student_id: str = "Qwen/Qwen3-1.7B-Base"
    teacher_id: str = "Qwen/Qwen3-4B-Instruct-2507"  # Modelo maestro grande (4B parámetros) que guía el entrenamiento

    # Prompting
    prompt_template: str = (
        "Solve step by step.\n"
        "Give ONLY ONE final numeric answer (no units), inside square brackets.\n"
        "Problem: {question}\n\nSolution:"
    )  # Plantilla de prompt para formatear las preguntas del dataset GSM8K
    max_new_tokens: int = 256  # Máximo número de tokens a generar en cada respuesta (afecta longitud de salida)

    # Generation temps
    eval_temperature: float = 0.0   # Temperatura para evaluación (0.0 = greedy decoding, determinista)
    train_temperature: float = 0.7  # Temperatura para generación durante entrenamiento (0.7 = sampling con diversidad)

    # Training schedule
    steps: int = 2  # Número total de iteraciones de entrenamiento
    batch_prompts: int = 2  # Número de prompts diferentes a muestrear en cada step
    samples_per_prompt: int = 3  # Número de muestras generadas por cada prompt (aumentar si tienes GPU grande)
    lr: float = 1e-4  # Learning rate del optimizador AdamW
    weight_decay: float = 0.0  # Regularización L2 (weight decay) - 0.0 significa sin regularización
    grad_accum: int = 1  # Número de pasos para acumular gradientes antes de hacer update (útil para simular batches grandes)

    # Micro-batching
    student_mb: int = 2  # Tamaño de micro-batch para procesar el modelo estudiante (reduce memoria GPU)

    # Monitoring
    log_every: int = 1  # Frecuencia de logging de métricas (cada N steps)
    val_every: int = 1  # Frecuencia de evaluación en validation set (cada N steps)
    val_sample_n: int = 10  # Número de ejemplos del validation set a usar en cada evaluación
    ema_momentum: float = 0.9  # Momentum para exponential moving average de métricas (suaviza fluctuaciones)

    # Validation size
    val_rows: Optional[int] = 10  # Tamaño del validation set (None = usar min(200, len(train)))

    # Output dir
    run_root: str = f"./run_opd_{int(time.time())}"  # Directorio raíz donde se guardan logs y checkpoints

cfg = Config()
os.makedirs(cfg.run_root, exist_ok=True)

def print_env():
    import transformers, accelerate, peft, matplotlib
    print("== Environment ==")
    print({
        "python": sys.version,
        "torch": torch.__version__,
        "transformers": transformers.__version__,
        "accelerate": accelerate.__version__,
        "peft": peft.__version__,
        "cuda": torch.version.cuda if torch.cuda.is_available() else "cpu",
        "device_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu",
        "platform": platform.platform(),
        "seed": SEED
    })
print_env()

== Environment ==
{'python': '3.12.12 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 20:05:38) [MSC v.1929 64 bit (AMD64)]', 'torch': '2.6.0+cu124', 'transformers': '4.51.3', 'accelerate': '1.4.0', 'peft': '0.14.0', 'cuda': '12.4', 'device_name': 'NVIDIA RTX 4000 Ada Generation Laptop GPU', 'platform': 'Windows-10-10.0.19045-SP0', 'seed': 42}


In [ ]:
# --------------------------
# Data: GSM8K
# --------------------------
def render_prompt(question: str) -> str:
    return cfg.prompt_template.format(question=question)

def parse_gold(answer_text: str) -> Optional[str]:
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", answer_text)
    if m: return m.group(1).strip()
    nums = re.findall(r"-?\d+(?:\.\d+)?", answer_text)
    return nums[-1].strip() if nums else None

def parse_pred(text: str) -> Optional[str]:
    m = re.search(r"\[\s*(-?\d+(?:\.\d+)?)\s*\]", text)
    if m: return m.group(1).strip()
    nums = re.findall(r"-?\d+(?:\.\d+)?", text)
    return nums[-1].strip() if nums else None

print("Loading GSM8K…")
ds_train_full = load_dataset("openai/gsm8k", "main", split="train")
ds_test       = load_dataset("openai/gsm8k", "main", split="test")

if cfg.val_rows is None:
    val_rows = min(200, len(ds_train_full))
else:
    val_rows = min(cfg.val_rows, len(ds_train_full))

ds_val   = ds_train_full.select(range(val_rows))
ds_train = ds_train_full.select(range(val_rows, len(ds_train_full)))

print(f"Splits: {len(ds_train)} train | {len(ds_val)} val | {len(ds_test)} test")


Loading GSM8K…
Splits: 7463 train | 10 val | 1319 test


In [ ]:
# --------------------------
# Tokenizers & Models
# --------------------------
def load_tokenizers(student_id: str, teacher_id: str):
    tok_s = AutoTokenizer.from_pretrained(student_id, use_fast=True)
    tok_t = AutoTokenizer.from_pretrained(teacher_id, use_fast=True)
    for tok in (tok_s, tok_t):
        if tok.pad_token is None and tok.eos_token is not None:
            tok.pad_token = tok.eos_token
        # para hacer inferencia, mejor que todos queden completos a la derecha asi se arranca desde la misma posicion siempre para generacion
        tok.padding_side = "left"  
    return tok_s, tok_t

def make_lora_student(model_id: str) -> torch.nn.Module:
    # Cargar explícitamente en CUDA
    base = AutoModelForCausalLM.from_pretrained(
        model_id, torch_dtype=torch.bfloat16, device_map="cuda"  # Cambiado de "auto" a "cuda"
    )
    base.config.use_cache = False  # off for training
    lora_cfg = LoraConfig(
        r=16, lora_alpha=32, lora_dropout=0.05, bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    )
    model = get_peft_model(base, lora_cfg)
    
    # Verificar que está en CUDA
    device = next(model.parameters()).device
    print(f"✓ Student model loaded on device: {device}")
    return model

def load_teacher(model_id: str) -> torch.nn.Module:
    # Cargar explícitamente en CUDA
    m = AutoModelForCausalLM.from_pretrained(
        model_id, torch_dtype=torch.bfloat16, device_map="cuda"  # Cambiado de "auto" a "cuda"
    ).eval()
    for p in m.parameters():
        p.requires_grad_(False)
    
    # Verificar que está en CUDA
    device = next(m.parameters()).device
    print(f"✓ Teacher model loaded on device: {device}")
    return m

tok_s, tok_t = load_tokenizers(cfg.student_id, cfg.teacher_id)

## Tests simples

In [8]:
# Para hacer un "subsample" de un Dataset y mantener el tipo Dataset (no dict):
ds_sample = ds_train.select(range(10))
ds_sample

Dataset({
    features: ['question', 'answer'],
    num_rows: 10
})

In [9]:
ds_sample[2]

{'question': 'Randy has 60 mango trees on his farm. He also has 5 less than half as many coconut trees as mango trees. How many trees does Randy have in all on his farm?',
 'answer': "Half of the number of Randy's mango trees is 60/2 = <<60/2=30>>30 trees.\nSo Randy has 30 - 5 = <<30-5=25>>25 coconut trees.\nTherefore, Randy has 60 + 25 = <<60+25=85>>85 treeson his farm.\n#### 85"}

In [10]:
# prompt renderizado
print(render_prompt(ds_sample[2]['question']))

Solve step by step.
Give ONLY ONE final numeric answer (no units), inside square brackets.
Problem: Randy has 60 mango trees on his farm. He also has 5 less than half as many coconut trees as mango trees. How many trees does Randy have in all on his farm?

Solution:


In [11]:
print(parse_gold(ds_sample[2]['answer']))

85


In [12]:
sample_solution = 'Creo que la solucion es [82]'
parse_pred(sample_solution)

'82'

In [41]:
# student
# model = make_lora_student(cfg.student_id)

# teacher
model = load_teacher(cfg.teacher_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✓ Teacher model loaded on device: cuda:0


In [17]:
# pruebas simples con distintos prompts para ver como siguen los modelos segun el prompt crudo o template o sistema, etc que le pasamos
prompt = 'My name is Qwen and my biggest problem is that'
prompt = "What do you know about Y-TEC?"
prompt = ds_sample[2]['question']
prompt = render_prompt(ds_sample[2]['question']) # como lo hacemos mas arriba
# prompt = f"""Answer the following question: {ds_sample[2]['question']}"""
# prompt = f"""User: Answer the following question: {ds_sample[2]['question']}
# Assistant: """
# prompt = f"""Question: {ds_sample[2]['question']}
# Answer: """

# ### formatear con el chat template de qwen (esto formatea automáticamente con <|im_start|> y <|im_end|>)
# messages = [{"role": "user", "content": ds_sample[2]['question']}]
# prompt = tok_t.apply_chat_template(
#     messages, 
#     tokenize=False,  # Si quieres solo el texto formateado
#     add_generation_prompt=True  # Agrega el prompt para que el modelo responda
# )
# ### 

k=4
temperature=0.0
max_new_tokens=100

model.eval(); model.config.use_cache = True
model_device = next(model.parameters()).device

enc = tok_s([prompt], return_tensors="pt").to(model_device)

gen_kwargs = dict(max_new_tokens=max_new_tokens, use_cache=True, pad_token_id=tok_s.pad_token_id, eos_token_id=tok_s.eos_token_id)
if temperature and temperature > 0.0:
        gen_kwargs.update(do_sample=True, temperature=temperature, top_p=0.9)
out = model.generate(**enc, **gen_kwargs)

text = tok_s.decode(out[0], skip_special_tokens=True)
print("="*80); print(f"Prompt raw:\n{prompt}"); print("="*80); print(f"Prompt raw + generacion del modelo:\n{text}")

Prompt raw:
Solve step by step.
Give ONLY ONE final numeric answer (no units), inside square brackets.
Problem: Randy has 60 mango trees on his farm. He also has 5 less than half as many coconut trees as mango trees. How many trees does Randy have in all on his farm?

Solution:
Prompt raw + generacion del modelo:
Solve step by step.
Give ONLY ONE final numeric answer (no units), inside square brackets.
Problem: Randy has 60 mango trees on his farm. He also has 5 less than half as many coconut trees as mango trees. How many trees does Randy have in all on his farm?

Solution: 60 + 5/2 - 5 = 60 + 2.5 - 5 = 57.5
Randy has 57.5 trees on his farm.


In [18]:
# TEST: evaluar

gold = parse_gold(ds_sample[2]['answer'])
print(f"Gold: {gold}")

pred = parse_pred(text) # esto es lo que sale del test de arriba
print(f"Pred: {pred}") # el parsing de la pred primero busca en brackets y si no hay, busca numeros

int(pred == gold)

Gold: 85
Pred: 57.5


0

## System

### Orquestrator -> json

In [105]:
# =========================
# Orchestrator: GraphSpec (2 agents) by TEXT
# =========================
from typing import Dict, Any, List, Optional
import json, re
import torch

# -------------------------
# 1) LLM client (usando modelo local)
# -------------------------
# Variables globales para el modelo y tokenizer (se asignan después de cargar el modelo)
_local_model = None
_local_tokenizer = None

def set_local_llm(model, tokenizer):
    """Configura el modelo y tokenizer locales para usar en llm_complete."""
    global _local_model, _local_tokenizer
    _local_model = model
    _local_tokenizer = tokenizer

@torch.no_grad()
def llm_complete(messages: List[Dict[str, str]], model: Optional[str] = None, temperature: float = 0.2, max_tokens: int = 800) -> str:
    """
    Usa el modelo local cargado en memoria para generar respuestas.
    Convierte los mensajes del formato chat a un prompt usando el chat template del tokenizer.
    """
    global _local_model, _local_tokenizer
    
    if _local_model is None or _local_tokenizer is None:
        raise ValueError("Modelo local no configurado. Llama a set_local_llm(model, tokenizer) primero.")
    
    # Preparar el modelo para inferencia
    _local_model.eval()
    _local_model.config.use_cache = True
    
    # Obtener el dispositivo del modelo
    model_device = next(_local_model.parameters()).device
    
    # Convertir mensajes a formato de prompt usando el chat template
    # Si el tokenizer tiene chat template, lo usamos; si no, construimos el prompt manualmente
    try:
        # Intentar usar el chat template del tokenizer
        prompt = _local_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    except (AttributeError, ValueError, TypeError):
        # Fallback: construir el prompt manualmente si no hay chat template
        prompt_parts = []
        for msg in messages:
            role = msg.get("role", "user")
            content = msg.get("content", "")
            if role == "system":
                prompt_parts.append(f"System: {content}\n")
            elif role == "user":
                prompt_parts.append(f"User: {content}\n")
            elif role == "assistant":
                prompt_parts.append(f"Assistant: {content}\n")
        prompt = "".join(prompt_parts) + "Assistant: "
    
    # Mostrar el prompt final
    print("\nPROMPT FINAL (input al modelo):")
    print(prompt)
    print()
    
    # Tokenizar el prompt
    enc = _local_tokenizer([prompt], return_tensors="pt").to(model_device)
    
    # Configurar parámetros de generación
    gen_kwargs = dict(
        max_new_tokens=max_tokens,
        use_cache=True,
        pad_token_id=_local_tokenizer.pad_token_id,
        eos_token_id=_local_tokenizer.eos_token_id
    )
    
    if temperature > 0.0:
        gen_kwargs.update(do_sample=True, temperature=temperature, top_p=0.9)
    else:
        gen_kwargs.update(do_sample=False)
    
    # Generar respuesta
    with torch.no_grad():
        out = _local_model.generate(**enc, **gen_kwargs)
    
    # Decodificar solo los tokens generados (sin el prompt)
    generated_tokens = out[0][enc['input_ids'].shape[1]:]
    text = _local_tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return text.strip()

# -------------------------
# 2) JSON extractor robusto
# -------------------------
def extract_json_from_text(text: str) -> Dict[str, Any]:
    """
    Busca el primer bloque ```json ... ``` o el primer {...} parseable.
    """
    print(f"{"="*50}\nRESPUESTA CRUDA DEL LLM:")
    print(text)
    print()
    
    # Try fenced block
    fence = re.search(r"```json\s*(\{.*?\})\s*```", text, re.S)
    if fence:
        json_str = fence.group(1)
        # print("✓ Encontrado bloque ```json```")
        # print("📋 JSON EXTRAÍDO:")
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"❌ Error al parsear JSON: {e}")
            raise
    
    # Try first balanced-ish { ... }
    brace = re.search(r"(\{.*\})", text, re.S)
    if brace:
        json_str = brace.group(1)
        # print("✓ Encontrado bloque {...}")
        # print("📋 JSON EXTRAÍDO:")
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"❌ Error al parsear JSON: {e}")
            raise
    
    print("❌ No se encontró JSON válido en el texto")
    raise ValueError("No se pudo extraer JSON del texto del orquestador.")

# -------------------------
# 3) Schema + validador mínimo (dos agentes)
# -------------------------
def validate_graph_spec(spec: Dict[str, Any]) -> None:
    """
    Validación para estructura simplificada (solo agents con system prompts).
    Estructura esperada:
    {
      "agents": [
        {
          "id": "agentA",
          "type": "llm",
          "system_prompt": "<string>"
        },
        {
          "id": "agentB",
          "type": "llm",
          "system_prompt": "<string>"
        }
      ]
    }
    """
    assert isinstance(spec, dict), "spec debe ser dict"
    assert "agents" in spec and isinstance(spec["agents"], list), "agents faltante"
    assert len(spec["agents"]) == 2, "v0: exactamente 2 agentes"
    ids = [a.get("id") for a in spec["agents"]]
    assert all(isinstance(i, str) for i in ids), "ids inválidos"
    assert len(set(ids)) == 2, "ids de agentes deben ser únicos"
    assert ids == ["agentA", "agentB"], "ids deben ser exactamente 'agentA' y 'agentB'"

    for a in spec["agents"]:
        assert a.get("type") == "llm", "v0: type debe ser 'llm'"
        assert isinstance(a.get("system_prompt"), str) and len(a["system_prompt"]) > 0, "system_prompt vacío"

# -------------------------
# 4) Prompt del orquestador (simplificado)
# -------------------------
ORCH_SYSTEM = (
    "You are designing a two-agent system to solve math word problems -> Agent A and Agent B."
    "The information flow is automatic: problem → Agent A → (problem + draft) → Agent B → final answer.\n\n"
    "Your task: Design appropriate system prompts for each agent that will guide them in their roles.\n"
    "Output ONLY a valid JSON object in a ```json block with this exact structure:\n"
    '{\n'
    '  "agents": [\n'
    '    {"id": "agentA", "type": "llm", "system_prompt": "<prompt for planning/drafting>"},\n'
    '    {"id": "agentB", "type": "llm", "system_prompt": "<prompt for solving/finalizing>"}\n'
    '  ]\n'
    '}\n\n'
    "Do not include anything else, only the structure above.\n"
    "The last agent is free to use any reasoning or format (including step-by-step) when working, but when giving the final_answer, it must write ONLY ONE final numeric answer (no units) inside square brackets.\n"
)

# -------------------------
# 5) Función principal del orquestador
# -------------------------
def orchestrator_generate_graph(problem_text: str, model: Optional[str] = None, temperature: float = 0.2, max_tokens: int = 800) -> Dict[str, Any]:
    """
    Llama al LLM con un prompt simple: system prompt + pregunta.
    Devuelve el dict ya parseado y validado.
    """
    messages = [
        {"role": "system", "content": ORCH_SYSTEM},
        {"role": "user", "content": problem_text},
    ]
    
    raw = llm_complete(messages, model=model, temperature=temperature, max_tokens=max_tokens)
    spec = extract_json_from_text(raw)
    validate_graph_spec(spec)
    return spec

In [106]:
# Configurar el modelo y tokenizer locales para el orquestador
# Asegúrate de que 'model' y 'tok_s' estén cargados antes de ejecutar esto
# Si usas el teacher model, cambia 'model' por 'teacher' y 'tok_s' por 'tok_t'
set_local_llm(model, tok_s)
print("✓ Modelo local configurado para el orquestador")


✓ Modelo local configurado para el orquestador


In [107]:
example_number = 3
print(ds_sample[example_number]['question'])
print('-'*40)
print(ds_sample[example_number]['answer'])

Jasper will serve charcuterie at his dinner party. He buys 2 pounds of cheddar cheese for $10, a pound of cream cheese that cost half the price of the cheddar cheese, and a pack of cold cuts that cost twice the price of the cheddar cheese. How much does he spend on the ingredients?
----------------------------------------
A pound of cream cheese cost $10 / 2 = $<<10/2=5>>5.
A pack of cold cuts cost $10 x 2 = $<<10*2=20>>20.
Jasper spent $10 + $5 + $20 = $<<10+5+20=35>>35 on the ingredients.
#### 35


In [113]:
temperature = 0.3
max_tokens = 500

mas_json = orchestrator_generate_graph(ds_sample[example_number]['question'], model, temperature, max_tokens)


PROMPT FINAL (input al modelo):
<|im_start|>system
You are designing a two-agent system to solve math word problems -> Agent A and Agent B.The information flow is automatic: problem → Agent A → (problem + draft) → Agent B → final answer.

Your task: Design appropriate system prompts for each agent that will guide them in their roles.
Output ONLY a valid JSON object in a ```json block with this exact structure:
{
  "agents": [
    {"id": "agentA", "type": "llm", "system_prompt": "<prompt for planning/drafting>"},
    {"id": "agentB", "type": "llm", "system_prompt": "<prompt for solving/finalizing>"}
  ]
}

Do not include anything else, only the structure above.
The last agent is free to use any reasoning or format (including step-by-step) when working, but when giving the final_answer, it must write ONLY ONE final numeric answer (no units) inside square brackets.
<|im_end|>
<|im_start|>user
Jasper will serve charcuterie at his dinner party. He buys 2 pounds of cheddar cheese for $10, a

In [114]:
mas_json

{'agents': [{'id': 'agentA',
   'type': 'llm',
   'system_prompt': 'You are tasked with creating a draft solution to a math word problem. First, identify all the given quantities and their relationships. Extract the prices of each ingredient based on the described relationships. Sum up the total cost of all ingredients in a clear, step-by-step draft. Do not provide a final answer yet; only generate a draft for the next agent to refine.'},
  {'id': 'agentB',
   'type': 'llm',
   'system_prompt': 'You receive a draft solution from Agent A. Review the draft, verify all calculations, and ensure the final answer is accurate. Perform the final computation to determine the total cost of the ingredients. Output only one numeric value in square brackets, with no units or explanations. For example: [45]'}]}

### Crear MAS a partir del json

In [115]:
# =========================
# Runner: ejecuta el GraphSpec (2 agentes en serie)
# =========================
from typing import Dict, Any, List, Optional

# Reusa llm_complete del bloque anterior.

def _render_user_inputs(input_map: Dict[str, Any]) -> str:
    """
    Render simple para el mensaje del 'user' a cada agente.
    """
    lines = []
    for k, v in input_map.items():
        lines.append(f"{k.upper()}: {v}")
    return "\n".join(lines)

def _agent_call_llm(agent: Dict[str, Any], input_payload: Dict[str, Any], temperature: float = 0.2, max_tokens: int = 512) -> str:
    """
    Construye mensajes (system + user) y llama al LLM.
    """
    sys_prompt = agent["system_prompt"]
    user_text = _render_user_inputs(input_payload)
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_text}
    ]
    return llm_complete(messages, model=None, temperature=temperature, max_tokens=max_tokens)

def run_two_agent_graph(graph_spec: Dict[str, Any], problem_text: str, temperature: float = 0.2, max_tokens: int = 512) -> Dict[str, Any]:
    """
    Ejecuta el grafo de 2 agentes en serie con flujo fijo.
    Flujo: problem → Agent A → (problem + draft) → Agent B → final answer

    Args:
        graph_spec: Dict con solo "agents" (lista de 2 agentes con id, type, system_prompt)
        problem_text: El problema matemático a resolver
        temperature: Temperatura para generación
        max_tokens: Máximo de tokens a generar

    Returns:
        Dict con 'trace' y 'final_text'
    """
    # Validación
    assert isinstance(graph_spec, dict), "graph_spec debe ser dict"
    assert "agents" in graph_spec and isinstance(graph_spec["agents"], list), "agents faltante"
    assert len(graph_spec["agents"]) == 2, "v0: exactamente 2 agentes"

    agent_a = graph_spec["agents"][0]
    agent_b = graph_spec["agents"][1]
    assert agent_a["id"] == "agentA", "Primer agente debe ser agentA"
    assert agent_b["id"] == "agentB", "Segundo agente debe ser agentB"

    trace: Dict[str, Any] = {"agents": {}}

    # Ejecutar Agent A: recibe solo el problema
    print("\n🤖 Ejecutando Agent A...")
    payload_a = {"problem": problem_text}
    draft = _agent_call_llm(agent_a, payload_a, temperature=temperature, max_tokens=max_tokens)
    print(f"Respuesta cruda Agent A:\n{draft}\n")  # <-- agregado para ver salida cruda

    trace["agents"]["agentA"] = {
        "system_prompt": agent_a["system_prompt"],
        "input_payload": payload_a,
        "output_text": draft
    }

    # Ejecutar Agent B: recibe problema + draft de A
    print("\n🤖 Ejecutando Agent B...")
    payload_b = {
        "problem": problem_text,
        "draft": draft
    }
    final_answer = _agent_call_llm(agent_b, payload_b, temperature=temperature, max_tokens=max_tokens)
    print(f"Respuesta cruda Agent B:\n{final_answer}\n")  # <-- agregado para ver salida cruda

    trace["agents"]["agentB"] = {
        "system_prompt": agent_b["system_prompt"],
        "input_payload": payload_b,
        "output_text": final_answer
    }

    return {"trace": trace, "final_text": final_answer}

In [ ]:
result = run_two_agent_graph(
    graph_spec=mas_json,
    problem_text=ds_sample[example_number]['question'],
    temperature=0.2,
    max_tokens=512
)


🤖 Ejecutando Agent A...

PROMPT FINAL (input al modelo):
<|im_start|>system
You are tasked with creating a draft solution to a math word problem. First, identify all the given quantities and their relationships. Extract the prices of each ingredient based on the described relationships. Sum up the total cost of all ingredients in a clear, step-by-step draft. Do not provide a final answer yet; only generate a draft for the next agent to refine.<|im_end|>
<|im_start|>user
PROBLEM: Jasper will serve charcuterie at his dinner party. He buys 2 pounds of cheddar cheese for $10, a pound of cream cheese that cost half the price of the cheddar cheese, and a pack of cold cuts that cost twice the price of the cheddar cheese. How much does he spend on the ingredients?<|im_end|>
<|im_start|>assistant


Respuesta cruda Agent A:
Let's break down the problem step by step:

1. **Cheddar cheese**:  
   Jasper buys 2 pounds of cheddar cheese for $10.  
   So, the total cost of cheddar cheese = **$10**



{'trace': {'agents': {'agentA': {'system_prompt': 'You are tasked with creating a draft solution to a math word problem. First, identify all the given quantities and their relationships. Extract the prices of each ingredient based on the described relationships. Sum up the total cost of all ingredients in a clear, step-by-step draft. Do not provide a final answer yet; only generate a draft for the next agent to refine.',
    'input_payload': {'problem': 'Jasper will serve charcuterie at his dinner party. He buys 2 pounds of cheddar cheese for $10, a pound of cream cheese that cost half the price of the cheddar cheese, and a pack of cold cuts that cost twice the price of the cheddar cheese. How much does he spend on the ingredients?'},
    'output_text': "Let's break down the problem step by step:\n\n1. **Cheddar cheese**:  \n   Jasper buys 2 pounds of cheddar cheese for $10.  \n   So, the total cost of cheddar cheese = **$10**\n\n2. **Cream cheese**:  \n   It costs **half the price of 

In [118]:
print(f"Output raw: {result['final_text']}")
print("="*30)
print(f"Output parsed: {parse_pred(result['final_text'])}")
print("="*30)
print(f"Gold: {parse_gold(ds_sample[example_number]['answer'])}")
print("="*30)
print(f"Correct: {int(parse_pred(result['final_text'])) == int(parse_gold(ds_sample[example_number]['answer']))}")

Output raw: [35]
Output parsed: 35
Gold: 35
Correct: True
